<a href="https://colab.research.google.com/github/f247805/thesis/blob/main/RobertA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

uploaded = files.upload()  # This will open a file upload dialog


Saving FINAL_CLEANED_CORRECTED_SHUFFLED_DATASET_NO_DUPLICATE.jsonl to FINAL_CLEANED_CORRECTED_SHUFFLED_DATASET_NO_DUPLICATE.jsonl


In [2]:
import os
os.listdir()


['.config',
 'FINAL_CLEANED_CORRECTED_SHUFFLED_DATASET_NO_DUPLICATE.jsonl',
 'sample_data']

In [3]:
import pandas as pd
df = pd.read_json("FINAL_CLEANED_CORRECTED_SHUFFLED_DATASET_NO_DUPLICATE.jsonl", lines=True)
df.head()


,text,labels
0,"the menu looked great, but the food was the bi...","[{'aspect': 'menu', 'polarity': 'positive'}, {..."
1,Pros : - Great sound for a $ 100 speaker at mi...,"[{'aspect': 'sound', 'polarity': 'positive'}]"
2,"Service was friendly, but not terribly attenti...","[{'aspect': 'Service', 'polarity': 'positive'}..."
3,"Gobo's scallion pancakes are a bit dry, and th...","[{'aspect': 'scallion pancakes', 'polarity': '..."
4,All apple associates are always wiling to help...,"[{'aspect': 'apple associates', 'polarity': 'p..."


In [5]:
!pip install transformers torch datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 824.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import torch
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification, MultiAspectRoBERTa,Trainer, TrainingArguments
from datasets import Dataset


ImportError: cannot import name 'MultiAspectRoBERTa' from 'transformers' (/usr/local/lib/python3.11/dist-packages/transformers/__init__.py)

In [ ]:
# Function to extract aspects and polarities
def extract_aspect_sentiment(labels):
    if isinstance(labels, list):  # Ensure it's a list
        aspects = [label.get("aspect", "").strip() for label in labels if isinstance(label, dict)]
        sentiments = [label.get("polarity", "").strip() for label in labels if isinstance(label, dict)]
    else:
        aspects, sentiments = [], []  # Handle missing or malformed data
    return aspects, sentiments

# Apply function to extract aspects & sentiments
df["aspects"], df["sentiments"] = zip(*df["labels"].apply(extract_aspect_sentiment))

# Check first few rows
print(df[["text", "aspects", "sentiments"]].head())


In [9]:
from sklearn.model_selection import train_test_split

# Extract sentences and aspect-polarity pairs
sentences = df["text"].tolist()
aspect_polarities = df["labels"].tolist()

# First, split into train (70%) and remaining (30%)
sentences_train, sentences_remaining, aspect_polarities_train, aspect_polarities_remaining = train_test_split(
    sentences, aspect_polarities, test_size=0.3, random_state=42
)

# Then, split remaining into validation (15%) and test (15%)
sentences_val, sentences_test, aspect_polarities_val, aspect_polarities_test = train_test_split(
    sentences_remaining, aspect_polarities_remaining, test_size=0.5, random_state=42
)

# Check dataset sizes
print(f"Train size: {len(sentences_train)}")
print(f"Validation size: {len(sentences_val)}")
print(f"Test size: {len(sentences_test)}")


Train size: 11288
Validation size: 2419
Test size: 2420


In [10]:
def tokenize_data(sentences, aspect_polarities):
    input_ids = []
    attention_masks = []

    for sentence, aspects in zip(sentences, aspect_polarities):
        for aspect in aspects:
            aspect_term = aspect["aspect"]

            encoding = tokenizer(
                sentence, aspect_term,  # Sentence + Aspect term
                padding="max_length",
                truncation=True,
                max_length=256,
                return_tensors="pt"
            )

            input_ids.append(encoding["input_ids"].squeeze())
            attention_masks.append(encoding["attention_mask"].squeeze())

    return torch.stack(input_ids), torch.stack(attention_masks)

# Tokenizing train, validation, and test sets
train_inputs, train_masks = tokenize_data(sentences_train, aspect_polarities_train)
val_inputs, val_masks = tokenize_data(sentences_val, aspect_polarities_val)
test_inputs, test_masks = tokenize_data(sentences_test, aspect_polarities_test)


In [11]:
# Mapping dictionary
label_mapping = {"positive": 2, "neutral": 1, "negative": 0}

# Function to convert aspect-polarity dictionaries into numeric labels
def convert_labels(aspect_polarities):
    return [
        [label_mapping[label["polarity"]] for label in polarity_list]
        for polarity_list in aspect_polarities
    ]

# Apply mapping to datasets
train_labels = convert_labels(aspect_polarities_train)
val_labels = convert_labels(aspect_polarities_val)
test_labels = convert_labels(aspect_polarities_test)


In [12]:
class ABSADataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_masks, labels, num_classes=3, max_aspects=5):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels
        self.num_classes = num_classes
        self.max_aspects = max_aspects

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_masks[idx],
        }

        label_ids = self.labels[idx]

        # Pad/truncate labels to fixed max_aspects
        if len(label_ids) < self.max_aspects:
            label_ids += [1] * (self.max_aspects - len(label_ids))  # Pad with "neutral" (1)
        else:
            label_ids = label_ids[:self.max_aspects]

        # Convert labels to a multi-hot tensor (One-hot for multi-label classification)
        multi_hot_labels = torch.zeros(self.max_aspects, self.num_classes)
        for i, label in enumerate(label_ids):
            multi_hot_labels[i, label] = 1

        item["labels"] = multi_hot_labels
        return item


Step 6: Convert Our Data into PyTorch Dataset

In [15]:
train_dataset = ABSADataset(train_inputs, train_masks, train_labels)
val_dataset = ABSADataset(val_inputs, val_masks, val_labels)
test_dataset = ABSADataset(test_inputs, test_masks, test_labels)


**Step 7: Create DataLoaders**

In [16]:
import torch
from torch.utils.data import DataLoader

BATCH_SIZE = 4  # Adjust as needed

def collate_fn(batch):
    input_ids = torch.stack([item["input_ids"] for item in batch])
    attention_mask = torch.stack([item["attention_mask"] for item in batch])

    labels = [item["labels"] for item in batch]  # List of tensors with different sizes

    # Determine the max number of aspects in this batch
    max_aspects = max(label.shape[0] for label in labels)

    # Create a padded label tensor (Batch_size, Max aspects, 3 classes)
    padded_labels = torch.zeros(len(batch), max_aspects, 3)

    for i, label in enumerate(labels):
        padded_labels[i, :label.shape[0], :] = label  # ✅ Corrected syntax and padding

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": padded_labels
    }

# ✅ Create PyTorch datasets
train_dataset = ABSADataset(train_inputs, train_masks, train_labels)
val_dataset = ABSADataset(val_inputs, val_masks, val_labels)
test_dataset = ABSADataset(test_inputs, test_masks, test_labels)

# ✅ Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)


In [19]:
class MultiAspectRoBERTa(nn.Module):
    def __init__(self, roberta, num_aspects, num_labels):
        super().__init__()
        self.roberta = roberta
        self.num_aspects = num_aspects  # ✅ Store num_aspects as an instance variable
        self.num_labels = num_labels  # ✅ Store num_labels as an instance variable
        hidden_size = roberta.config.hidden_size
        self.fc = nn.Linear(hidden_size, num_aspects * num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # [CLS] token
        logits = self.fc(pooled_output)
        logits = logits.view(-1, self.num_aspects, self.num_labels)  # ✅ Use self.num_aspects & self.num_labels
        return logits


NameError: name 'nn' is not defined

In [18]:
from transformers import AutoModel  # <-- Add this line

# Step 1: Create an instance of the model
roberta = AutoModel.from_pretrained("roberta-base")  # Load pretrained RoBERTa model
model = MultiAspectRoBERTa(roberta, num_aspects=5, num_labels=3)  # Pass required arguments

# Step 2: Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Now this will work


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NameError: name 'MultiAspectRoBERTa' is not defined

In [ ]:
from transformers import AdamW
from transformers import get_scheduler  # <-- Add this line

optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss function
criterion = nn.BCEWithLogitsLoss()


# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)

# Learning rate scheduler
num_training_steps = len(train_loader) * 3  # Assuming 3 epochs
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)


1. Define Loss Function

In [ ]:
from torch.nn import CrossEntropyLoss

loss_fn = CrossEntropyLoss()


Training Loop

In [ ]:
import torch
import torch.nn.functional as F

def evaluate(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels.float())

            total_loss += loss.item()

            # Compute accuracy per aspect
            preds = torch.argmax(outputs, dim=-1)  # Get class predictions per aspect
            correct += (preds == torch.argmax(labels, dim=-1)).sum().item()
            total += torch.numel(preds)

    avg_loss = total_loss / len(val_loader)
    accuracy = correct / total
    return avg_loss, accuracy


In [ ]:
num_epochs = 5  # You can adjust this
best_val_loss = float("inf")  # Track the best validation loss

for epoch in range(num_epochs):
    model.train()  # Set to training mode
    total_loss = 0

    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()  # Reset gradients
        ooutputs = model(input_ids, attention_mask)  # Now: [batch_size, max_aspects, num_classes]
        loss = criterion(ooutputs, labels.float())  # Ensure labels are float for BCEWithLogitsLoss


        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {avg_train_loss:.4f}")

    # **Run validation at the end of each epoch**
    val_loss, val_accuracy = evaluate(model, val_loader, criterion, device)
    print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}")

    # **Save model if validation loss improves**
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pth")
        print("Model saved!")

print("Training complete!")


In [ ]:
print(outputs.shape)  # Should be (batch_size, max_aspects, num_classes)
print(labels.shape)

In [ ]:
!nvidia-smi
!while true; do nvidia-smi; sleep 5; done
